In [31]:
from rich.console import Console
from openai import OpenAI
from dotenv import load_dotenv
import json

load_dotenv(override=True)

True

In [32]:
# handle_tool_calls function
# loop function
# call loop

In [33]:
openai = OpenAI()

In [34]:
def show(text):
    try:
        Console().print(text)
    except Exception:
        print(text)

In [35]:
todos, completed = [], []

In [36]:
def get_todo_report() -> str:
    show("\n".join([f"{i+1}. {todo} {'[X]' if completed[i] else '[ ]'}" for i, todo in enumerate(todos)]))


In [37]:
def create_todos(descriptions: list[str]) -> str:
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return get_todo_report()

def mark_complete(index: int, completion_notes: str) -> str:
    if index < 0 or index >= len(todos):
        return "Invalid index"
    completed[index] = True
    Console().print(completion_notes)
    return get_todo_report()

In [38]:
create_todos_json = {
    "name": "create_todos",
    "description": "Create a list of todos",
    "parameters": {
        "type": "object",
        "properties": {
            "descriptions": {"type": "array", "items": {"type": "string"}}
        },
        "required": ["descriptions"],
        "additionalProperties": False
    }
}

In [39]:
mark_complete_json = {
    "name": "mark_complete",
    "description": "Mark a todo as complete",
    "parameters": {
        "type": "object",
        "properties": {
            "index": {"type": "integer"},
            "completion_notes": {"type": "string"}
        },
        "required": ["index"],
        "additionalProperties": False
    }
}

In [40]:
tools = [{
    "type": "function",
    "function": create_todos_json
}, {
    "type": "function",
    "function": mark_complete_json
}]

def handle_tool_calls(tool_calls: list[dict]) -> list[dict]:
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id})
    return results

In [41]:
def loop(messages):
    done = False
    while not done:
        response = openai.chat.completions.create(model="gpt-5.2", messages=messages, tools=tools, reasoning_effort="none")
        finish_reason = response.choices[0].finish_reason
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    show(response.choices[0].message.content)
            

In [42]:
system_message = """
You are given a problem to solve, by using your todo tools to plan a list of steps, then carrying out each step in turn.
Now use the todo list tools, create a plan, carry out the steps, and reply with the solution.
If any quantity isn't provided in the question, then include a step to come up with a reasonable estimate.
Provide your solution in Rich console markup without code blocks.
Do not ask the user questions or clarification; respond only with the answer after using your tools.
"""
user_message = """
Suppose you are planning a party for 18 people.
You want to order enough pizza so that everyone gets at least 3 slices, and each pizza has 8 slices. 
Additionally, you need to buy drinks so that everyone gets at least two 355ml cans. 
If cans are only sold in 12-packs, and pizzas can be ordered individually, how many pizzas and 12-packs of drinks should you buy? 
Provide your calculations and final answer.
"""
messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_message}]



In [43]:
todos, completed = [], []
loop(messages)

1. Compute total pizza slices needed (18 people * 3 slices) and convert to pizzas of 8 slices, rounding up to whole
pizzas. [ ]
2. Compute total drink cans needed (18 people * 2 cans) and convert to 12-packs, rounding up to whole packs. [ ]
3. Summarize calculations and provide final quantities in Rich console markup. [ ]

Slices needed: 18*3=54. Pizzas: 54/8=6.75 -> round up to 7 pizzas (56 slices).

1. Compute total pizza slices needed (18 people * 3 slices) and convert to pizzas of 8 slices, rounding up to whole
pizzas. [X]
2. Compute total drink cans needed (18 people * 2 cans) and convert to 12-packs, rounding up to whole packs. [ ]
3. Summarize calculations and provide final quantities in Rich console markup. [ ]

Cans needed: 18*2=36. 12-packs: 36/12=3 exactly -> 3 packs (36 cans).

1. Compute total pizza slices needed (18 people * 3 slices) and convert to pizzas of 8 slices, rounding up to whole
pizzas. [X]
2. Compute total drink cans needed (18 people * 2 cans) and convert to 12-packs, rounding up to whole packs. [X]
3. Summarize calculations and provide final quantities in Rich console markup. [ ]

Prepare final response with calculations and final counts.

1. Compute total pizza slices needed (18 people * 3 slices) and convert to pizzas of 8 slices, rounding up to whole
pizzas. [X]
2. Compute total drink cans needed (18 people * 2 cans) and convert to 12-packs, rounding up to whole packs. [X]
3. Summarize calculations and provide final quantities in Rich console markup. [X]

Pizza  
- People: 18  
- Minimum slices per person: 3  
- Total slices needed: 18 × 3 = 54 slices  
- Slices per pizza: 8  
- Pizzas needed: 54 ÷ 8 = 6.75 → round up to 7 pizzas (7 × 8 = 56 slices)

Drinks  
- Minimum cans per person: 2 (355 ml each)  
- Total cans needed: 18 × 2 = 36 cans  
- Cans per 12-pack: 12  
- 12-packs needed: 36 ÷ 12 = 3

Final order  
- 7 pizzas  
- 3 twelve-packs of drinks